## 把input根據aspect term拆成左右兩邊丟掉bert，去預測polarity ---> 失敗

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

### 對處理好的laptop、restaurant的train、test資料作前處理

In [2]:
#把dataframe裡的text切成text左邊跟右邊的function
def split_text(df):
    df['left_text'] = 'N/A'
    df['right_text'] = 'N/A'
    
    for i in tqdm(range(len(df))):
        text = df.loc[i, 'text']
        aspect = df.loc[i, 'aspect']
        text_split = text.split(aspect) # 根據aspect切割text左右邊
        
        df.loc[i,'left_text'] = text_split[0]+aspect
        df.loc[i,'right_text'] = aspect+text_split[1]
        
    return df

In [3]:
laptop_train = pd.read_csv('dataset/laptop_train_processed.csv', encoding='utf-8')
restaurant_train = pd.read_csv('dataset/restaurant_train_processed.csv', encoding='utf-8')
laptop_test = pd.read_csv('dataset/laptop_test_processed.csv', encoding='utf-8')
restaurant_test = pd.read_csv('dataset/restaurant_test_processed.csv', encoding='utf-8')

# 把train的資料串在一起且切割text
train_data = laptop_train.append(restaurant_train)
train_data = train_data.reset_index(drop=True)
train_data = split_text(train_data)

# laptop_test切割text
laptop_test = split_text(laptop_test)

#restaurant_test切割text
restaurant_test = split_text(restaurant_test)

#把test的資料串在一起且切割text
# test_data = laptop_test.append(restaurant_test)
# test_data = test_data.reset_index(drop=True)
# test_data = split_text(test_data)

laptop_test.head(10)

100%|██████████| 1120/1120 [00:00<00:00, 3986.52it/s]


,text,aspect,polarity,left_text,right_text
0,"Boot time is super fast, around anywhere from ...",Boot time,positive,Boot time,"Boot time is super fast, around anywhere from ..."
1,tech support would not fix the problem unless ...,tech support,negative,tech support,tech support would not fix the problem unless ...
2,Set up was easy.,Set up,positive,Set up,Set up was easy.
3,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative,Did not enjoy the new Windows 8,Windows 8 and touchscreen functions.
4,Did not enjoy the new Windows 8 and touchscree...,touchscreen functions,negative,Did not enjoy the new Windows 8 and touchscree...,touchscreen functions.
5,Other than not being a fan of click pads (indu...,internal speakers,negative,Other than not being a fan of click pads (indu...,"internal speakers, it's hard for me to find th..."
6,Other than not being a fan of click pads (indu...,price tag,positive,Other than not being a fan of click pads (indu...,price tag.
7,Other than not being a fan of click pads (indu...,click pads,negative,Other than not being a fan of click pads,click pads (industry standard these days) and ...
8,No installation disk (DVD) is included.,installation disk (DVD),neutral,No installation disk (DVD),installation disk (DVD) is included.
9,"It's fast, light, and simple to use.",use,positive,"It's fast, light, and simple to use",use.


In [4]:
# print一個出來看看
print(train_data.loc[0,'left_text'])
print(train_data.loc[0,'right_text'])

I charge it at night and skip taking the cord
cord with me because of the good battery life.


In [5]:
#看train跟test的資料每一個label各是多少個
train_data_positive = train_data[train_data['polarity'] == 'positive']
train_data_negative = train_data[train_data['polarity'] == 'negative']
train_data_neutral = train_data[train_data['polarity'] == 'neutral']

laptop_test_positive = laptop_test[laptop_test['polarity'] == 'positive']
laptop_test_negative = laptop_test[laptop_test['polarity'] == 'negative']
laptop_test_neutral = laptop_test[laptop_test['polarity'] == 'neutral']

restaurant_test_positive = restaurant_test[restaurant_test['polarity'] == 'positive']
restaurant_test_negative = restaurant_test[restaurant_test['polarity'] == 'negative']
restaurant_test_neutral = restaurant_test[restaurant_test['polarity'] == 'neutral']

print('      ','pos','neg','neutral')
print('train', len(train_data_positive), len(train_data_negative), len(train_data_neutral))
print('laptop', len(laptop_test_positive), len(laptop_test_negative), len(laptop_test_neutral))
print('restau', len(restaurant_test_positive), len(restaurant_test_negative), len(restaurant_test_neutral))

       pos neg neutral
train 3151 1671 1093
laptop 341 128 169
restau 728 196 196


In [6]:
#把polarity變成數字的label，positive是2，neutral是1，negative是0
train_data.loc[train_data['polarity']=='positive', 'label'] = 2
train_data.loc[train_data['polarity']=='negative', 'label'] = 0
train_data.loc[train_data['polarity']=='neutral', 'label'] = 1
train_data['label'] = train_data['label'].astype(int)

laptop_test.loc[laptop_test['polarity']=='positive', 'label'] = 2
laptop_test.loc[laptop_test['polarity']=='negative', 'label'] = 0
laptop_test.loc[laptop_test['polarity']=='neutral', 'label'] = 1
laptop_test['label'] = laptop_test['label'].astype(int)

restaurant_test.loc[restaurant_test['polarity']=='positive', 'label'] = 2
restaurant_test.loc[restaurant_test['polarity']=='negative', 'label'] = 0
restaurant_test.loc[restaurant_test['polarity']=='neutral', 'label'] = 1
restaurant_test['label'] = restaurant_test['label'].astype(int)

laptop_test.head(10)

,text,aspect,polarity,left_text,right_text,label
0,"Boot time is super fast, around anywhere from ...",Boot time,positive,Boot time,"Boot time is super fast, around anywhere from ...",2
1,tech support would not fix the problem unless ...,tech support,negative,tech support,tech support would not fix the problem unless ...,0
2,Set up was easy.,Set up,positive,Set up,Set up was easy.,2
3,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative,Did not enjoy the new Windows 8,Windows 8 and touchscreen functions.,0
4,Did not enjoy the new Windows 8 and touchscree...,touchscreen functions,negative,Did not enjoy the new Windows 8 and touchscree...,touchscreen functions.,0
5,Other than not being a fan of click pads (indu...,internal speakers,negative,Other than not being a fan of click pads (indu...,"internal speakers, it's hard for me to find th...",0
6,Other than not being a fan of click pads (indu...,price tag,positive,Other than not being a fan of click pads (indu...,price tag.,2
7,Other than not being a fan of click pads (indu...,click pads,negative,Other than not being a fan of click pads,click pads (industry standard these days) and ...,0
8,No installation disk (DVD) is included.,installation disk (DVD),neutral,No installation disk (DVD),installation disk (DVD) is included.,1
9,"It's fast, light, and simple to use.",use,positive,"It's fast, light, and simple to use",use.,2


In [7]:
#找出最多字的cell是幾個字的function，用來判斷bert的維度要用多少
def find_max_word(df):
    max_count = 0
    for i in tqdm(range(len(df))):
        left_text_word_count = len(df.loc[i,'left_text'].split())
        right_text_word_count = len(df.loc[i,'right_text'].split())
        big_count = max(left_text_word_count, right_text_word_count)
        if big_count>max_count:
            max_count = big_count
    return max_count

In [8]:
# 找出left text跟right text中字數最多的是幾個字
train_max_count = find_max_word(train_data)
laptop_test_max_count = find_max_word(laptop_test)
restaurant_test_max_count = find_max_word(restaurant_test)
print('訓練資料集字數最多是:',train_max_count)
print('laptop測試資料集字數最多是:',laptop_test_max_count)
print('restaurant測試資料集字數最多是:',restaurant_test_max_count)

100%|██████████| 1120/1120 [00:00<00:00, 87059.07it/s]

訓練資料集字數最多是: 72
laptop測試資料集字數最多是: 60
restaurant測試資料集字數最多是: 54


### Bert轉資料小小測試一下

In [9]:
import tensorflow as tf
from transformers import BertTokenizer, BertModel, TFBertForSequenceClassification, TFBertModel

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten, InputLayer
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import concatenate

In [10]:
# Load pre-trained model tokenizer, to convert our text into tokens that correspond to BERT’s vocabulary.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
print(train_data.loc[4,'text'])

The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.


In [12]:
# 把句子切成一個一個單字或符號
text = train_data.loc[4, 'text']
print(text)
tokens = tokenizer.tokenize(text)      # 每個字切詞成一個list
print(type(tokens))                 # list
np.array(tokens)                    # 轉成numpy

The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.
<class 'list'>


array(['the', 'tech', 'guy', 'then', 'said', 'the', 'service', 'center',
       'does', 'not', 'do', '1', '-', 'to', '-', '1', 'exchange', 'and',
       'i', 'have', 'to', 'direct', 'my', 'concern', 'to', 'the', '"',
       'sales', '"', 'team', ',', 'which', 'is', 'the', 'retail', 'shop',
       'which', 'i', 'bought', 'my', 'net', '##book', 'from', '.'],
      dtype='<U8')

In [13]:
# 把英文單字或符號轉換成數字token，成為token list
input_ids = tokenizer.convert_tokens_to_ids(tokens)   # 每個字轉成id
print(type(input_ids))                         # list
print(len(input_ids))
np.array(input_ids)

<class 'list'>
44


array([1996, 6627, 3124, 2059, 2056, 1996, 2326, 2415, 2515, 2025, 2079,
       1015, 1011, 2000, 1011, 1015, 3863, 1998, 1045, 2031, 2000, 3622,
       2026, 5142, 2000, 1996, 1000, 4341, 1000, 2136, 1010, 2029, 2003,
       1996, 7027, 4497, 2029, 1045, 4149, 2026, 5658, 8654, 2013, 1012])

In [14]:
# 把token list加上 CLS、SEP(起始跟結束的token)
input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
print(type(input_ids))
print(len(input_ids))
np.array(input_ids)
# 101是CLS，101是SEP

<class 'list'>
46


array([ 101, 1996, 6627, 3124, 2059, 2056, 1996, 2326, 2415, 2515, 2025,
       2079, 1015, 1011, 2000, 1011, 1015, 3863, 1998, 1045, 2031, 2000,
       3622, 2026, 5142, 2000, 1996, 1000, 4341, 1000, 2136, 1010, 2029,
       2003, 1996, 7027, 4497, 2029, 1045, 4149, 2026, 5658, 8654, 2013,
       1012,  102])

In [15]:
#把每一個input句子都設成128維度，不夠補0
input_dim = 128 #預設維度
n = input_dim - len(input_ids)
input_ids2 = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))  # array右邊append n 個 0  補長度到128
print(len(input_ids2))
input_ids2

128


array([ 101, 1996, 6627, 3124, 2059, 2056, 1996, 2326, 2415, 2515, 2025,
       2079, 1015, 1011, 2000, 1011, 1015, 3863, 1998, 1045, 2031, 2000,
       3622, 2026, 5142, 2000, 1996, 1000, 4341, 1000, 2136, 1010, 2029,
       2003, 1996, 7027, 4497, 2029, 1045, 4149, 2026, 5658, 8654, 2013,
       1012,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

### 把上述處理建立一個function，丟入一串文字試試 

In [16]:
# 把維度固定在128維
input_dim = 128
def input_ids_all(text):
#     if len(text) > input_dim-2:
#         text = text[0:126]        
    tokens = tokenizer.tokenize(text)                          # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)        # 每個字轉成id
    input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
    input_ids = np.array(input_ids)                            # list 轉 numpy
    if len(input_ids) < input_dim:
        n = input_dim - len(input_ids)
        input_ids = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))    # array右邊append n 個 0  補長度到512
    return input_ids

In [17]:
# text = 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.'
print(text)
input_ids_all(text)

The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.


array([ 101, 1996, 6627, 3124, 2059, 2056, 1996, 2326, 2415, 2515, 2025,
       2079, 1015, 1011, 2000, 1011, 1015, 3863, 1998, 1045, 2031, 2000,
       3622, 2026, 5142, 2000, 1996, 1000, 4341, 1000, 2136, 1010, 2029,
       2003, 1996, 7027, 4497, 2029, 1045, 4149, 2026, 5658, 8654, 2013,
       1012,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

### 正式把資料餵進模型

In [18]:
#轉成numpy
train_left_text = train_data['left_text'].to_numpy()
train_right_text = train_data['right_text'].to_numpy()

laptop_left_text = laptop_test['left_text'].to_numpy()
laptop_right_text = laptop_test['right_text'].to_numpy()

restaurant_left_text = restaurant_test['left_text'].to_numpy()
restaurant_right_text = restaurant_test['right_text'].to_numpy()

print('train_left_text資料數:',len(train_left_text))
print('train_right_text資料數:',len(train_right_text))
print('laptop_left_text資料數:',len(laptop_left_text))
print('laptop_right_text資料數:',len(laptop_right_text))
print('restaurant_left_text資料數:',len(restaurant_left_text))
print('restaurant_right_text資料數:',len(restaurant_right_text))
print()
print('第一筆train_left_text:',train_left_text[0])
print('資料型態:',type(train_left_text))

train_left_text資料數: 5915
train_right_text資料數: 5915
laptop_left_text資料數: 638
laptop_right_text資料數: 638
restaurant_left_text資料數: 1120
restaurant_right_text資料數: 1120

第一筆train_left_text: I charge it at night and skip taking the cord
資料型態: <class 'numpy.ndarray'>


In [19]:
# 把train、laptop_test、restaurant_test 的input改成token、numpy格式
train_left_input_ids = [input_ids_all(i) for i in train_left_text]      # 必須要 [ ] 輸出是list
train_left_input_ids = np.array(train_left_input_ids)                    # 轉成numpy

train_right_input_ids = [input_ids_all(i) for i in train_right_text]
train_right_input_ids = np.array(train_right_input_ids)

laptop_left_input_ids = [input_ids_all(i) for i in laptop_left_text]
laptop_left_input_ids = np.array(laptop_left_input_ids)

laptop_right_input_ids = [input_ids_all(i) for i in laptop_right_text]
laptop_right_input_ids = np.array(laptop_right_input_ids)

restaurant_left_input_ids = [input_ids_all(i) for i in restaurant_left_text]
restaurant_left_input_ids = np.array(restaurant_left_input_ids)

restaurant_right_input_ids = [input_ids_all(i) for i in restaurant_right_text]
restaurant_right_input_ids = np.array(restaurant_right_input_ids)

print('train_left_input_ids資料數:',len(train_left_input_ids))
print('train_right_input_ids資料數:',len(train_right_input_ids))
print('laptop_left_input_ids資料數:',len(laptop_left_input_ids))
print('laptop_right_input_ids資料數:',len(laptop_right_input_ids))
print('restaurant_left_input_ids資料數:',len(restaurant_left_input_ids))
print('restaurant_right_input_ids資料數:',len(restaurant_right_input_ids))
print()
print(train_left_input_ids[4])
print(train_right_input_ids[4])

train_left_input_ids資料數: 5915
train_right_input_ids資料數: 5915
laptop_left_input_ids資料數: 638
laptop_right_input_ids資料數: 638
restaurant_left_input_ids資料數: 1120
restaurant_right_input_ids資料數: 1120

[ 101 1996 6627 3124  102    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[ 101 6627 3124 2059 2056 1996 2326 2415 2515 2025 2079 1015 1011 2000
 1011 1015 3863 1998 1045 2031 2000 3622 2026 5142 2000 1996 1000 4341
 1000 2136 10

In [20]:
# 把train、laptop_test、restaurant_test 的label改成numpy
train_label = train_data['label'].to_numpy()
laptop_label = laptop_test['label'].to_numpy()
restaurant_label = restaurant_test['label'].to_numpy()
print(train_label)
print(len(train_label))
print(len(laptop_label))
print(len(restaurant_label))

[1 2 0 ... 1 1 1]
5915
638
1120


In [47]:
# 檢查polarity跟label有沒有不一樣
for i in range(10):
    print(laptop_test.loc[i, 'polarity'], laptop_test.loc[i, 'label'], laptop_label[i])
print()
for i in range(11):
    print(restaurant_test.loc[i, 'polarity'], restaurant_test.loc[i, 'label'], restaurant_label[i])

positive 2 2
negative 0 0
positive 2 2
negative 0 0
negative 0 0
negative 0 0
positive 2 2
negative 0 0
neutral 1 1
positive 2 2

positive 2 2
positive 2 2
positive 2 2
positive 2 2
positive 2 2
positive 2 2
positive 2 2
positive 2 2
positive 2 2
positive 2 2
neutral 1 1


### 建立Model

In [48]:
# first input model
input_layer_1 = Input(shape = (128,), dtype='int64')
bert_model_1 = TFBertModel.from_pretrained('bert-base-uncased')(input_layer_1)
bert_model_1 = bert_model_1[0]
dropout_1 = Dropout(0.1)(bert_model_1)
flat1 = Flatten()(bert_model_1)

# second input model
input_layer_2 = Input(shape = (128,), dtype='int64')
bert_model_2 = TFBertModel.from_pretrained('bert-base-uncased')(input_layer_2)
bert_model_2 = bert_model_2[0]
dropout = Dropout(0.1)(bert_model_2)
flat2 = Flatten()(bert_model_2)


merge = concatenate([flat1, flat2]) # merge input model
output = Dense(units=3, activation = 'softmax')(merge)
model = Model(inputs=[input_layer_1, input_layer_2], outputs=output)
print(model.summary())

optimizer = tf.keras.optimizers.Adam(learning_rate=7e-6, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(loss=loss, optimizer=optimizer, metrics=[metric])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_2 (TFBertModel)   ((None, 128, 768), ( 109482240   input_3[0][0]                    
__________________________________________________________________________________________________
tf_bert_model_3 (TFBertModel)   ((None, 128, 768), ( 109482240   input_4[0][0]                    
____________________________________________________________________________________________

In [49]:
# train_data、laptop_test
model_fit = model.fit([train_left_input_ids,train_right_input_ids], train_label, 
                      batch_size=4, epochs=5, 
                      validation_data=([laptop_left_input_ids,laptop_right_input_ids], laptop_label)
#                      steps_per_epoch=115,
#                      validation_steps=7)
                   )

Train on 5915 samples, validate on 638 samples
Epoch 1/5
5915/5915 [==============================] - 196s 33ms/sample - loss: 1.0192 - accuracy: 0.5319 - val_loss: 1.0170 - val_accuracy: 0.5345
Epoch 2/5
5915/5915 [==============================] - 176s 30ms/sample - loss: 1.0187 - accuracy: 0.5327 - val_loss: 1.0170 - val_accuracy: 0.5345
Epoch 3/5
5915/5915 [==============================] - 176s 30ms/sample - loss: 1.0187 - accuracy: 0.5327 - val_loss: 1.0169 - val_accuracy: 0.5345
Epoch 4/5
5915/5915 [==============================] - 176s 30ms/sample - loss: 1.0188 - accuracy: 0.5327 - val_loss: 1.0170 - val_accuracy: 0.5345
Epoch 5/5
1612/5915 [=======>......................] - ETA: 2:03 - loss: 1.0079 - accuracy: 0.5435

KeyboardInterrupt: 

In [50]:
# train_data、restaurant_test
model_fit = model.fit([train_left_input_ids,train_right_input_ids], train_label, 
                      batch_size=4, epochs=5, 
                      validation_data=([restaurant_left_input_ids,restaurant_right_input_ids], restaurant_label)
#                      steps_per_epoch=115,
#                      validation_steps=7)
                   )

Train on 5915 samples, validate on 1120 samples
Epoch 1/5
5915/5915 [==============================] - 180s 30ms/sample - loss: 1.0187 - accuracy: 0.5327 - val_loss: 0.9014 - val_accuracy: 0.6500
Epoch 2/5
5915/5915 [==============================] - 178s 30ms/sample - loss: 1.0187 - accuracy: 0.5327 - val_loss: 0.9014 - val_accuracy: 0.6500
Epoch 3/5
5152/5915 [=========================>....] - ETA: 21s - loss: 1.0204 - accuracy: 0.5299

KeyboardInterrupt: 